In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import PolynomialFeatures
import re
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()

db_user = os.getenv('DB_USER')
db_pass = os.getenv('DB_PASS')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_name = os.getenv('DB_NAME')

In [4]:
engine = create_engine(f"postgresql+psycopg2://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}")
df_games = pd.read_sql("SELECT * FROM games", engine)

In [5]:
# Helper function to sort rounds
def get_round_order(row):
    # Match regular rounds: 'R1', 'R23', etc.
    match = re.match(r'R(\d+)$', row['round'])
    if match:
        return int(match.group(1))
    # Finals rounds mapping (AFL convention)
    finals_order = {
        'REF': 100,  # Elimination Final (week 1)
        'RQF': 101,  # Qualifying Final (week 1)
        'RSF': 102,  # Semi Final (week 2)
        'RPF': 103,  # Preliminary Final (week 3)
        'RGF': 104,  # Grand Final (week 4)
    }
    code = row['round'][1:] 
    return finals_order.get(code, 999)  # Unknown finals get 999

In [6]:
# Create a dataframe so we can calculate player movement between rounds
query_movement ="""
SELECT 
    pgs.game_id, 
    g.season_year, 
    g.round, 
    pgs.team_id, 
    pgs.player_id
FROM player_game_stats pgs
JOIN games g ON pgs.game_id = g.id
ORDER BY g.season_year, g.round, pgs.team_id
"""
df_players = pd.read_sql(query_movement, engine)

players_per_team_round = (
    df_players
    .groupby(['season_year', 'team_id', 'round'])
    .agg({'player_id': lambda x: set(x)})
    .reset_index()
    .sort_values(['season_year', 'team_id', 'round'])
)

# Use the helper function to sort the rounds
players_per_team_round['round_order'] = players_per_team_round.apply(get_round_order, axis=1)
players_per_team_round = players_per_team_round.sort_values(
    ['season_year', 'team_id', 'round_order']
).reset_index(drop=True)

# Once we have the dataframe, now we need to sort it and calculate how many players played together in previous round, plus previous 2 & 3 rounds
players_per_team_round['prev_players_1'] = (
    players_per_team_round.groupby(['season_year', 'team_id'])['player_id'].shift(1)
)
players_per_team_round['prev_players_2'] = (
    players_per_team_round.groupby(['season_year', 'team_id'])['player_id'].shift(2)
)
players_per_team_round['prev_players_3'] = (
    players_per_team_round.groupby(['season_year', 'team_id'])['player_id'].shift(3)
)

# Functions to calculate number of changes vs previous 1, 2, 3 rounds
def num_changes_vs_prev(row):
    if pd.isna(row['prev_players_1']):
        return np.nan
    return len(row['player_id'].symmetric_difference(row['prev_players_1']))

def num_changes_vs_prev2(row):
    if pd.isna(row['prev_players_1']) or pd.isna(row['prev_players_2']):
        return np.nan
    prev_union = row['prev_players_1'].union(row['prev_players_2'])
    return len(row['player_id'].symmetric_difference(prev_union))

def num_changes_vs_prev3(row):
    if pd.isna(row['prev_players_1']) or pd.isna(row['prev_players_2']) or pd.isna(row['prev_players_3']):
        return np.nan
    prev_union = row['prev_players_1'].union(row['prev_players_2']).union(row['prev_players_3'])
    return len(row['player_id'].symmetric_difference(prev_union))

players_per_team_round['num_changes_1'] = players_per_team_round.apply(num_changes_vs_prev, axis=1)
players_per_team_round['num_changes_2'] = players_per_team_round.apply(num_changes_vs_prev2, axis=1)
players_per_team_round['num_changes_3'] = players_per_team_round.apply(num_changes_vs_prev3, axis=1)

# Optional: Display the first few rows
print(players_per_team_round[['season_year', 'team_id', 'round', 'num_changes_1', 'num_changes_2', 'num_changes_3']].head())

   season_year  team_id round  num_changes_1  num_changes_2  num_changes_3
0         2010        5    R1            NaN            NaN            NaN
1         2010        5    R2            2.0            NaN            NaN
2         2010        5    R3            6.0            7.0            NaN
3         2010        5    R4            6.0            7.0            8.0
4         2010        5    R5            6.0            3.0            5.0


In [7]:
# Calculate the average team age
query_age = """
SELECT 
    pgs.game_id, 
    pgs.team_id, 
    pgs.player_id, 
    p.birth_date, 
    g.game_date
FROM player_game_stats pgs
JOIN games g ON pgs.game_id = g.id
JOIN players p ON pgs.player_id = p.id
"""
df_players_age = pd.read_sql(query_age, engine)

df_players_age['age'] = (
    (pd.to_datetime(df_players_age['game_date']) - pd.to_datetime(df_players_age['birth_date'])).dt.days / 365.25
)

avg_age_per_team_game = (
    df_players_age
    .groupby(['game_id', 'team_id'])['age']
    .mean()
    .reset_index()
    .rename(columns={'age': 'avg_age', 'game_id': 'id'}) 
)


In [8]:
# calculate days between games as well as rolling averages of previous 5 games
df_games['game_date'] = pd.to_datetime(df_games['game_date'])

df_home = df_games[['id', 'season_year', 'round', 'game_date', 'home_team_id', 'home_score_total', 'away_score_total']].copy()
df_home = df_home.rename(columns={
    'home_team_id': 'team_id',
    'home_score_total': 'score_scored',
    'away_score_total': 'score_conceded'
})

df_away = df_games[['id', 'season_year', 'round', 'game_date', 'away_team_id', 'away_score_total', 'home_score_total']].copy()
df_away = df_away.rename(columns={
    'away_team_id': 'team_id',
    'away_score_total': 'score_scored',
    'home_score_total': 'score_conceded'
})

df_team_games = pd.concat([df_home, df_away], ignore_index=True)
df_team_games = df_team_games.sort_values(['team_id', 'game_date']).reset_index(drop=True)

# calculate average score and margin based on rolling average of previous 5 games, minimum 1 game and only including games in season
df_team_games['avg_scored_prev5'] = (
    df_team_games.groupby(['team_id', 'season_year'])['score_scored']
    .transform(lambda x: x.shift(1).rolling(5, min_periods=1).mean())
)

df_team_games['avg_conceded_prev5'] = (
    df_team_games.groupby(['team_id', 'season_year'])['score_conceded']
    .transform(lambda x: x.shift(1).rolling(5, min_periods=1).mean())
)

df_team_games['margin'] = df_team_games['score_scored'] - df_team_games['score_conceded']

df_team_games['avg_margin_prev5'] = (
    df_team_games.groupby(['team_id', 'season_year'])['margin']
    .transform(lambda x: x.shift(1).rolling(5, min_periods=1).mean())
)

# calculate days since previous game, only including games in season
df_team_games['days_since_prev'] = (
    df_team_games.groupby(['team_id', 'season_year'])['game_date']
    .diff()
    .dt.days
)

In [9]:
# merge everything together into single dataframe
team_game_features = (
    pd.merge(
        df_team_games,
        players_per_team_round[['season_year', 'team_id', 'round', 'num_changes_1', 'num_changes_2', 'num_changes_3']],
        on=['season_year', 'team_id', 'round'],
        how='left'
    )
    .merge(
        avg_age_per_team_game,
        on=['id', 'team_id'],
        how='left'
    )
)

In [10]:
# Get win/loss results

df_home_result = df_games[['id', 'season_year', 'round', 'home_team_id', 'home_result']]
df_home_result = df_home_result.rename(columns={
    'id': 'game_id',
    'home_team_id': 'team_id',
    'home_result': 'result'
})
df_home_result['home_or_away'] = 'home'

df_away_result = df_games[['id', 'season_year', 'round', 'away_team_id', 'away_result']]
df_away_result = df_away_result.rename(columns={
    'id': 'game_id',
    'away_team_id': 'team_id',
    'away_result': 'result'
})
df_away_result['home_or_away'] = 'away'

df_results = pd.concat([df_home_result, df_away_result], ignore_index=True)

In [11]:
# Merge results with what we want to analyse on
team_game_features = team_game_features.merge(
    df_results[['game_id', 'team_id', 'result', 'home_or_away']],
    left_on=['id', 'team_id'],
    right_on=['game_id', 'team_id'],
    how='left'
).drop(columns=['game_id'])

team_game_features['target_win'] = team_game_features['result'].map({'W': 1, 'L': 0, 'D': np.nan})

In [12]:
# Add home ground advantage
team_game_features['is_home'] = (team_game_features['home_or_away'] == 'home').astype(int)

In [44]:
# Prepare features and target
features = [
    'avg_scored_prev5', 
    'avg_conceded_prev5', 
    'days_since_prev', 
    'num_changes_2',
    'avg_age',
    'is_home'
]
analysis_df = team_game_features.dropna(subset=['target_win'] + features)

# Create the model
X = analysis_df[features] 
y = analysis_df['target_win']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

model = LogisticRegression(max_iter=1000)  
model.fit(X_scaled, y)

LogisticRegression(max_iter=1000)

In [45]:
# Check coefficients
coefs = pd.Series(model.coef_[0], index=features)
print("Logistic Regression Coefficients:")
print(coefs.sort_values())

Logistic Regression Coefficients:
avg_conceded_prev5   -0.229053
num_changes_2        -0.083847
days_since_prev      -0.030959
avg_age               0.270478
avg_scored_prev5      0.272462
is_home               0.312109
dtype: float64


In [46]:
# Check accuracy of model
y_pred = model.predict(X_scaled)
acc = accuracy_score(y, y_pred)
print(f"Accuracy: {acc:.3f} ({acc*100:.1f}%)")

Accuracy: 0.617 (61.7%)


In [16]:
# One drop method to see biggest impact of dropping a variable
baseline_auc = roc_auc_score(y, model.predict_proba(X_scaled)[:, 1])
print("Baseline AUC:", baseline_auc)

for i, feature in enumerate(features):
    X_reduced = np.delete(X_scaled, i, axis=1)
    model_temp = LogisticRegression(max_iter=1000)
    model_temp.fit(X_reduced, y)
    auc = roc_auc_score(y, model_temp.predict_proba(X_reduced)[:, 1])
    print(f"AUC without {feature}: {auc:.3f} (Drop: {baseline_auc - auc:.3f})")

Baseline AUC: 0.664629780937198
AUC without avg_scored_prev5: 0.644 (Drop: 0.020)
AUC without num_changes_2: 0.662 (Drop: 0.003)
AUC without avg_age: 0.640 (Drop: 0.025)
AUC without is_home: 0.643 (Drop: 0.022)


In [17]:
# Let's look at whether multiple factors together can predict a winner
poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
X_poly = poly.fit_transform(X_scaled)
feature_names = poly.get_feature_names_out(features)
model_poly = LogisticRegression(max_iter=1000)
model_poly.fit(X_poly, y)

coefs_poly = pd.Series(model_poly.coef_[0], index=feature_names)
# Show the biggest absolute coefficients (most "important" effects)
print(coefs_poly.abs().sort_values(ascending=False).head(20))

avg_age                           0.349069
is_home                           0.306232
avg_scored_prev5                  0.297511
num_changes_2                     0.123351
avg_scored_prev5 is_home          0.064229
avg_scored_prev5 avg_age          0.039819
num_changes_2 is_home             0.039793
avg_scored_prev5 num_changes_2    0.031703
num_changes_2 avg_age             0.010428
avg_age is_home                   0.004427
dtype: float64


In [18]:
# Evaluate predictive accuracy
y_pred_poly = model_poly.predict(X_poly)
y_prob_poly = model_poly.predict_proba(X_poly)[:,1]
print("Poly Accuracy:", accuracy_score(y, y_pred_poly))
print("Poly AUC:", roc_auc_score(y, y_prob_poly))

Poly Accuracy: 0.6174795300818797
Poly AUC: 0.666004222558408


In [19]:
corr = team_game_features[['num_changes_1', 'num_changes_2', 'num_changes_3', 'avg_scored_prev5', 'avg_conceded_prev5']].corr()
print(corr)

                    num_changes_1  num_changes_2  num_changes_3  \
num_changes_1            1.000000       0.825097       0.726724   
num_changes_2            0.825097       1.000000       0.890943   
num_changes_3            0.726724       0.890943       1.000000   
avg_scored_prev5        -0.208813      -0.272864      -0.317988   
avg_conceded_prev5       0.227175       0.300940       0.352887   

                    avg_scored_prev5  avg_conceded_prev5  
num_changes_1              -0.208813            0.227175  
num_changes_2              -0.272864            0.300940  
num_changes_3              -0.317988            0.352887  
avg_scored_prev5            1.000000           -0.236927  
avg_conceded_prev5         -0.236927            1.000000  
